In [1]:
import numpy as np
import nidaqmx
from tqdm import tqdm

In [2]:
# Define the input and output channels
input_channel = "PXI1Slot2/ai0"
# output_channels = ["PXI1Slot3/ao0", "PXI1Slot3/ao2", "PXI1Slot3/ao4", "PXI1Slot3/ao6"]
output_channels = ["PXI1Slot3/ao{_i}".format(_i=i) for i in range(4)]

# Define the SPGD parameters
learning_rate = 5
scale = 0.4
output_scale=1
n_channels = len(output_channels)

In [3]:
def check_range(v,scale):
    idx = np.where(np.abs(v) + scale > 9.5)
    if idx[0].size > 0:
        v[idx[0]] = 5
    return v

In [5]:
with nidaqmx.Task() as input_task, nidaqmx.Task() as output_task:
    
    # Configure the input task
    input_task.ai_channels.add_ai_voltage_chan(input_channel)

    # Configure the output task
    output_task.ao_channels.add_ao_voltage_chan(",".join(output_channels))
    v = 2*np.random.normal(0.2,size=(n_channels,))#2*np.ones(4)

    for _ in tqdm(range(10000),ncols=120):
    # while True:
        delta_v = scale*np.random.rand(n_channels)

        v = check_range(v,scale)

        # print('reading..')
        output_task.write(v,auto_start=True)
        # output_task.wait_until_done()
        input_signal_v = output_scale*input_task.read()

        # print('positive perturbation')
        pos_v =v+delta_v#check_range(v+delta_v)
        output_task.write(pos_v,auto_start=True)
        # output_task.wait_until_done()
        input_signal_plus = output_scale*input_task.read()

        # print('negative perturbation')
        neg_v = v-delta_v#check_range(v-delta_v)
        output_task.write(neg_v,auto_start=True)
        # output_task.wait_until_done()
        input_signal_minus = output_scale*input_task.read()

        gradient = (input_signal_plus - input_signal_minus) * delta_v

        v += learning_rate*gradient
        # print('grad:',gradient.mean(), 'v:',v)


100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:14<00:00, 671.47it/s]


In [6]:
# with nidaqmx.Task() as input_task, nidaqmx.Task() as output_task:
    
#     # Configure the input task
#     input_task.ai_channels.add_ai_voltage_chan(input_channel)

#     # Configure the output task
#     output_task.ao_channels.add_ao_voltage_chan(",".join(output_channels))
#     v = 9.7*np.ones(4)

#     # for _ in tqdm(range(100),ncols=120):
#     while True:
#         delta_v = 1*np.random.rand(4)

#         v = check_range(v)

#         # print('reading..')
#         output_task.write(v,auto_start=True)
#         output_task.wait_until_done()
#         input_signal_v = input_task.read()

#         # print('positive perturbation')
#         pos_v =check_range(v+delta_v)
#         output_task.write(pos_v,auto_start=True)
#         output_task.wait_until_done()
#         input_signal_plus = input_task.read()

#         # print('negative perturbation')
#         neg_v = check_range(v-delta_v)
#         output_task.write(neg_v,auto_start=True)
#         output_task.wait_until_done()
#         input_signal_minus = 10*input_task.read()

#         gradient = (input_signal_plus - input_signal_minus) * delta_v

#         v += learning_rate*gradient
#         print('grad:',gradient, 'v:',v)
